In [6]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model, StatsTracker
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES, get_bt_index

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import encode_target, FGW
from mol_opt.train_mol_opt import ft, load_checkpoint, save_checkpoint, get_latest_model

from rdkit.Chem import MolFromSmiles

from molgen.metrics.Penalty import Penalty

import torch
from torch import nn
import numpy as np
import time
import shutil

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def rename(oldname, newname):
    wd = "mol_opt/output_dev2/"

    olddir = os.path.join(wd, oldname)
    newdir = os.path.join(wd, newname)

    shutil.copytree(olddir, newdir)

    for fl in os.listdir(newdir):
    #     print (fl)
        newfl = fl.replace(oldname, newname)
        os.rename(os.path.join(wd, newname, fl), os.path.join(wd, newname, newfl))
        
    wd = "mol_opt/logs_dev2/"
    olddir = os.path.join(wd, oldname)
    newdir = os.path.join(wd, newname)

    shutil.copytree(olddir, newdir)

In [53]:
oldname = "pointwise-base-nopen-4000-gumbel4"
oldname = "pointwise-base-longer-softmax3"
newname = "pointwise-base-longer-softmax"
rename(oldname, newname)

In [25]:
oldname = "pointwise-base-nopen-4000-softmax4"
newname = "pointwise-base-longer-softmax3"
rename(oldname, newname)

In [56]:
wd = "mol_opt/output_dev2/"
newname = "pointwise-base-longer-softmax3"
filename = os.path.join(wd, newname, "model_" + newname + "_12009")


('model_pointwise-base-longer-softmax3_12009', 12009)

In [98]:
wd = "mol_opt/output_dev2/"
newname = "pointwise-base-longer-softmax3"
filename, _ = get_latest_model(newname, os.path.join(wd, newname))
print (filename)
molopt, molopt_decoder, optimizer, penalty, scheduler, args, epoch = load_checkpoint(os.path.join(wd, newname, filename))

args.connectivity = True 
args.valency = True 
args.euler_characteristic_penalty = True
args.conn_lambda_start = 1
args.conn_lambda_end = 30
args.conn_lambda_epochs_start = 12000
args.conn_lambda_epochs_end = 14000
args.valency_lambda_start = 10
args.valency_lambda_end = 50
args.valency_lambda_epochs_start = 12000
args.valency_lambda_epochs_end = 14000
args.euler_lambda_start = 1
args.euler_lambda_end = 20 
args.euler_lambda_epochs_start = 12000
args.euler_lambda_epochs_end = 14000
args.tau_start = 1
args.tau_end = 0.001
args.tau_epochs_start = 1
args.tau_epochs_end = 16000


penalty.conn_lambda = args.conn_lambda_start
penalty.valency_lambda = args.valency_lambda_start
penalty.euler_lambda = args.euler_lambda_start

save_checkpoint(molopt, molopt_decoder, optimizer, penalty, scheduler, epoch, args, filename)
molopt, molopt_decoder, optimizer, penalty, scheduler, args, epoch = load_checkpoint(filename)
penalty.log()
penalty.euler_lambda_epochs_start
# penalty.connectivity
penalty.tau_epochs_end
print(penalty.conn_lambda_rate)
print(penalty.euler_lambda_rate)

model_pointwise-base-longer-softmax3_12089
Penalty params: tau=0.00980 conn_l=1.00000 val_l=10.00000 euler_l=1.00000 epoch=12089 mode=[1 1 1] conn=False euler=False val=False
1.0017020455288321
1.0014989884985714


In [100]:
1.08281/1.08097

1.001702174898471

In [101]:
1.05201/1.05043

1.0015041459211942

In [4]:
wd = "mol_opt/output_dev2/"
oldname = "pointwise-base-nopen-4000-gumbel4"
dirname = os.path.join(wd, oldname)

In [15]:
os.listdir(dirname)

['model_pointwise-base-nopen-4000-gumbel4_1000',
 'model_pointwise-base-nopen-4000-gumbel4_2000',
 'model_pointwise-base-nopen-4000-gumbel4_3000',
 'model_pointwise-base-nopen-4000-gumbel4_3100',
 'model_pointwise-base-nopen-4000-gumbel4_3200',
 'model_pointwise-base-nopen-4000-gumbel4_3300',
 'model_pointwise-base-nopen-4000-gumbel4_3400',
 'model_pointwise-base-nopen-4000-gumbel4_3500',
 'model_pointwise-base-nopen-4000-gumbel4_3600',
 'model_pointwise-base-nopen-4000-gumbel4_3700',
 'model_pointwise-base-nopen-4000-gumbel4_3800',
 'model_pointwise-base-nopen-4000-gumbel4_3900',
 'model_pointwise-base-nopen-4000-gumbel4_3910',
 'model_pointwise-base-nopen-4000-gumbel4_3920',
 'model_pointwise-base-nopen-4000-gumbel4_3930',
 'model_pointwise-base-nopen-4000-gumbel4_3940',
 'model_pointwise-base-nopen-4000-gumbel4_3950',
 'model_pointwise-base-nopen-4000-gumbel4_3960',
 'model_pointwise-base-nopen-4000-gumbel4_3970',
 'model_pointwise-base-nopen-4000-gumbel4_3980',
 'model_pointwise-ba

In [8]:
_, lastepoch = get_latest_model(oldname, dirname)

In [14]:
for fl in os.listdir(dirname):
    ep = int(fl.split("_")[2])
    ep_diff = lastepoch - ep
    for modulo in [10, 100, 1000]:
        if ep_diff > modulo and ep % modulo != 0:
            os.remove(os.path.join(dirname, fl))